In [2]:
import os
import io
from rich import print
from dotenv import load_dotenv
from datetime import date
from pydantic import BaseModel
from fastapi import FastAPI, UploadFile, File
from openai import OpenAI
import firebase_admin
from firebase_admin import credentials, firestore
from prompts import (ENRICHED_GOAL_SYS_MSG,
                     ENRICHED_GOAL_USR_MSG,
                     GOAL_TO_MILESTONE_SYS_MSG,
                     GOAL_TO_MILESTONE_USR_MSG,
                     MILESTONE_TO_TASK_SYS_MSG, 
                     MILESTONE_TO_TASK_USR_MSG)

In [3]:
# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

cred_path = ("firebase_credentials.json")
cred = credentials.Certificate(cred_path)
firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
class Task(BaseModel):
    id: str
    name: str
    completed: bool
    duration_hours: float
#    date: date
    goal_id: str
    milestone_id: str

class Milestone(BaseModel):
    id: str
    name: str
    duration_weeks: float
    completed: bool
    goal_id: str

class GoalMilestones(BaseModel):
    milestones: list[Milestone]

class WeeklyTasks(BaseModel):
    tasks: list[Task]


class MilestonesGeneration:

    def __init__(self, user_id: str, goal_id: str):

        self.user_id = user_id
        self.goal_id = goal_id

        # Fetch user profile data
        user_profile_docs = db.collection("users").document(user_id).collection("userProfile").stream()
        user_profile_doc = next(user_profile_docs, None)  # Get the first document
        self.user_profile_data = user_profile_doc.to_dict() if user_profile_doc else {}

        # Fetch goal data
        goal_doc = db.collection("users", self.user_id, "goals").document(self.goal_id).get()
        self.goal_data = goal_doc.to_dict() if goal_doc.exists else {}


    def enrich_goal(self):

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages = [
                {"role": "system", "content": ENRICHED_GOAL_SYS_MSG},
                {"role": "user", "content": ENRICHED_GOAL_USR_MSG.replace("$GOAL",str(self.goal_data)).replace("$PROFILE", str(self.user_profile_data))}
            ],
            seed=42
        )

        return completion.choices[0].message.content




    def generate_milestones(self, augmented_context: str = "None"):

        # Logic to generate tasks based on a prompt and augmented context
        completion = client.beta.chat.completions.parse(
            model='gpt-4o-2024-08-06',
            messages=[
                {"role": "system", "content": GOAL_TO_MILESTONE_SYS_MSG},
                {"role": "user", "content": GOAL_TO_MILESTONE_USR_MSG.replace("$GOAL", self.enrich_goal())}
                       ],
            response_format=GoalMilestones,
            seed=42
        )
        
        return completion.choices[0].message.parsed

In [5]:
class TasksGeneration:

    def __init__(self, user_id: str, goal_id: str, milestone_data: str):

        self.user_id = user_id
        self.goal_id = goal_id
        #self.milestone_id = milestone_id

        # Fetch user profile data
        user_profile_docs = db.collection("users").document(user_id).collection("userProfile").stream()
        user_profile_doc = next(user_profile_docs, None)  # Get the first document
        self.user_profile_data = user_profile_doc.to_dict() if user_profile_doc else {}

        # Fetch goal data
        goal_doc = db.collection("users", self.user_id, "goals").document(self.goal_id).get()
        self.goal_data = goal_doc.to_dict() if goal_doc.exists else {}

        self.milestone_data = milestone_data


    def generate_tasks(self, augmented_context: str = "None"):

        # Logic to generate tasks based on a prompt and augmented context
        completion = client.beta.chat.completions.parse(
            model='gpt-4o-2024-08-06',
            messages=[
                {"role": "system", "content": MILESTONE_TO_TASK_SYS_MSG},
                {"role": "user", "content": MILESTONE_TO_TASK_USR_MSG.replace("$MILESTONE",str(self.milestone_data))
                       .replace("$GOAL", str(self.goal_data))}
                       ],
            response_format=WeeklyTasks,
            seed=42
        )
        
        return completion.choices[0].message.parsed

In [7]:
print(TasksGeneration("saIj3tndzESNUy2jO5iU5SMKFU73", "qXJvA5OU0nHcSj9f0gsj").enrich_goal())

Bybysker aims to solve five hard Leetcode problems and three system design interview questions within a 1-3 month 
timeframe. This goal is designed to enhance his coding skills, a key development opportunity for him, and aligns 
with his long-term vision of becoming a reference in data science and AI. With available time of 10-20 hours 
weekly, Bybysker can allocate 3-5 hours for focused learning and practice, leveraging his adaptability and 
multimodal learning style to manage challenges such as time management and procrastination. The progress will be 
measured by the number of problems solved and the speed at which easy, medium, and hard problems are tackled on 
Leetcode. Achieving this goal is relevant to his aspirations of spending time in a FAANG company, contributing to 
his dream of being a successful serial entrepreneur in Africa and Europe. His past achievement of being a 
valedictorian highlights his ability to commit and excel, providing a solid foundation for this current challenge.

In [16]:
print(TasksGeneration("saIj3tndzESNUy2jO5iU5SMKFU73", "qXJvA5OU0nHcSj9f0gsj").generate_tasks())

DailyTasks(
    tasks=[
        Task(id='1', title='Join a System Design Workshop', completed=False, goal_id='1'),
        Task(
            id='2',
            title='Engage in a 1-hour Coding Session for the Leetcode problem (Hard)',
            completed=False,
            goal_id='2'
        ),
        Task(id='3', title='Watch a 30-minute Tutorial on System Design Basics', completed=False, goal_id='3'),
        Task(id='4', title='Create a Weekly Schedule to Improve Time Management', completed=False, goal_id='4'),
        Task(
            id='5',
            title='Attend a 15-minute Mindfulness Session for Focus and Stress Reduction',
            completed=False,
            goal_id='5'
        )
    ]
)

In [6]:
milestonesss = MilestonesGeneration("saIj3tndzESNUy2jO5iU5SMKFU73", "qXJvA5OU0nHcSj9f0gsj").generate_milestones()
print(milestonesss)

GoalMilestones(
    milestones=[
        Milestone(
            id='MS1',
            name='Setup Strategy and Initial Benchmarking',
            duration_weeks=2.0,
            completed=False,
            goal_id='G1'
        ),
        Milestone(
            id='MS2',
            name='Focused Practice on Problem Solving',
            duration_weeks=4.0,
            completed=False,
            goal_id='G1'
        ),
        Milestone(
            id='MS3',
            name='Full System Design Interviews',
            duration_weeks=3.0,
            completed=False,
            goal_id='G1'
        ),
        Milestone(
            id='MS4',
            name='Consolidation and Retrospective',
            duration_weeks=3.0,
            completed=False,
            goal_id='G1'
        )
    ]
)

In [7]:
print(milestonesss.milestones)#[0])

[
    Milestone(
        id='MS1',
        name='Setup Strategy and Initial Benchmarking',
        duration_weeks=2.0,
        completed=False,
        goal_id='G1'
    ),
    Milestone(
        id='MS2',
        name='Focused Practice on Problem Solving',
        duration_weeks=4.0,
        completed=False,
        goal_id='G1'
    ),
    Milestone(id='MS3', name='Full System Design Interviews', duration_weeks=3.0, completed=False, goal_id='G1'),
    Milestone(
        id='MS4',
        name='Consolidation and Retrospective',
        duration_weeks=3.0,
        completed=False,
        goal_id='G1'
    )
]

In [15]:
tasksss1 = TasksGeneration("saIj3tndzESNUy2jO5iU5SMKFU73", "qXJvA5OU0nHcSj9f0gsj", milestonesss.milestones[0]).generate_tasks()
tasksss2 = TasksGeneration("saIj3tndzESNUy2jO5iU5SMKFU73", "qXJvA5OU0nHcSj9f0gsj", milestonesss.milestones[1]).generate_tasks()
tasksss3 = TasksGeneration("saIj3tndzESNUy2jO5iU5SMKFU73", "qXJvA5OU0nHcSj9f0gsj", milestonesss.milestones[2]).generate_tasks()

In [16]:
print(tasksss1)

WeeklyTasks(
    tasks=[
        Task(
            id='T1',
            name='Research and Identify Resources',
            completed=False,
            duration_hours=2.0,
            goal_id='FAANG-Goal',
            milestone_id='1'
        ),
        Task(
            id='T2',
            name='Plan the Study Schedule',
            completed=False,
            duration_hours=1.5,
            goal_id='FAANG-Goal',
            milestone_id='1'
        ),
        Task(
            id='T3',
            name='Set Up Tools and Accounts',
            completed=False,
            duration_hours=1.0,
            goal_id='FAANG-Goal',
            milestone_id='1'
        ),
        Task(
            id='T4',
            name='Define Metrics for Problem Solving Progress',
            completed=False,
            duration_hours=0.5,
            goal_id='FAANG-Goal',
            milestone_id='1'
        ),
        Task(
            id='T5',
            name='Join a Relevant Study Group or Community',
            completed=False,
            duration_hours=1.0,
            goal_id='FAANG-Goal',
            milestone_id='1'
        )
    ]
)

In [17]:
print(tasksss2)

WeeklyTasks(
    tasks=[
        Task(
            id='2.1',
            name='Research common coding interview problems',
            completed=False,
            duration_hours=2.0,
            goal_id='FAANG-Goal',
            milestone_id='2'
        ),
        Task(
            id='2.2',
            name='Solve one hard Leetcode problem',
            completed=False,
            duration_hours=3.0,
            goal_id='FAANG-Goal',
            milestone_id='2'
        ),
        Task(
            id='2.3',
            name='Watch system design interview examples',
            completed=False,
            duration_hours=2.0,
            goal_id='FAANG-Goal',
            milestone_id='2'
        ),
        Task(
            id='2.4',
            name='Draft a system design for a simple application',
            completed=False,
            duration_hours=3.0,
            goal_id='FAANG-Goal',
            milestone_id='2'
        ),
        Task(
            id='2.5',
            name='Review solutions and identify areas for improvement',
            completed=False,
            duration_hours=1.0,
            goal_id='FAANG-Goal',
            milestone_id='2'
        )
    ]
)

In [18]:
print(tasksss3)

WeeklyTasks(
    tasks=[
        Task(
            id='3.1',
            name='Solve 3 medium-level Leetcode problems',
            completed=False,
            duration_hours=4.0,
            goal_id='FAANG-Goal',
            milestone_id='3'
        ),
        Task(
            id='3.2',
            name='Complete 2 Leetcode-style timed practice assessments',
            completed=False,
            duration_hours=3.0,
            goal_id='FAANG-Goal',
            milestone_id='3'
        ),
        Task(
            id='3.3',
            name='Participate in 1 online coding competition',
            completed=False,
            duration_hours=3.0,
            goal_id='FAANG-Goal',
            milestone_id='3'
        ),
        Task(
            id='3.4',
            name='Study system design principles for 2 hours',
            completed=False,
            duration_hours=2.0,
            goal_id='FAANG-Goal',
            milestone_id='3'
        ),
        Task(
            id='3.5',
            name='Join a system design study group and discuss 1 problem per week',
            completed=False,
            duration_hours=2.0,
            goal_id='FAANG-Goal',
            milestone_id='3'
        )
    ]
)